In [1]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2025-02-11 19:14:08--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6 [following]
--2025-02-11 19:14:08--  https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc39de5f947633aa1afa49cae4e6.dl.dropboxusercontent.com/cd/0/inline/Cj8LDGpuVHCRHnCB_XRBpk3W-HhXuJBhhPjUdwB2AMpABcvGtPDUlg4AVkiAghG1oxxf4MOL7_8ThE_l_FyLfK7s1AqlK1SZvJ1v1ItD8dnrPurpxgwoTH6cqUu1ghlokkI/file# [following]
--2025-02-11 19:14:08--  https://uc39de5f947633aa1afa49cae4

In [3]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

In [4]:
display(data_df.head())
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


In [5]:
data_df = data_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

In [6]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())
display(data_df.clean_label.unique())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [7]:
data_df = data_df.drop_duplicates("input", keep="first")
data_df = data_df.reset_index(drop=True)
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ,service
freq,1,2111


In [8]:
# remove class `contact` as it has only 4 sample (cannot do train 80 val 10 test 10)
data_df = data_df[data_df.clean_label != 'contact']

In [9]:
data = data_df.copy().to_numpy()

unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

print("Create Mappings")
display(num_2_label_map)
display(label_2_num_map)

# print("Before Mappings")
# display(data[:, 1])
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'officer'}

{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'officer': 24}

In [10]:
def strip_str(string):
    return string.strip()
     
# print("Before")
# print(data)
data[:,0] = np.vectorize(strip_str)(data[:,0])
# print("After")
# print(data)

# Model 2 MUSE

Build a simple logistic regression model using features from the MUSE model.

Which MUSE model will you use? Why?

**Ans:**
`sentence-transformers/use-cmlm-multilingual` (the first link) simply because it is more popular and has more downloads. Thus, it is maintained more frequently, has more support and should be more stable.

MUSE is typically used with tensorflow. However, there are some pytorch conversions made by some people.

- https://huggingface.co/sentence-transformers/use-cmlm-multilingual
- https://huggingface.co/dayyass/universal-sentence-encoder-multilingual-large-3-pytorch

In [11]:
import time
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
data

array([['<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 7],
       ['ยอดเงินในระบบ', 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 1]], dtype=object)

In [14]:
sentences = data[:,0]
len(sentences)

13385

In [15]:
sentences = data[:,0]
model = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')
embeddings = model.encode(sentences, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Some weights of the model checkpoint at sentence-transformers/use-cmlm-multilingual were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/419 [00:00<?, ?it/s]

In [16]:
print(embeddings.shape)

(13385, 768)


In [17]:
X = embeddings.copy()
y = data[:, 1]

print(X.shape, y.shape)

(13385, 768) (13385,)


In [21]:
X_train: np.ndarray
y_train: np.ndarray
X_val: np.ndarray
y_val: np.ndarray
X_test: np.ndarray
y_test: np.ndarray

X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
y_train = y_train.astype(int)
y_test = y_test.astype(int)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(10708, 768) (10708,) (1338, 768) (1338,) (1339, 768) (1339,)


In [24]:
# 500 max epochs took 2s
logmodel = LogisticRegression(class_weight='balanced', max_iter=500, random_state=2025)
logmodel.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=2025)

In [25]:
start_time = time.time()
predictions = logmodel.predict(X_test)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Total time: 0.009260177612304688


In [26]:
# 500 max epochs acc 64.001%
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 64.002987%


In [27]:
report = classification_report(y_test, predictions, output_dict=True, digits=2)
report

{'0': {'precision': 0.5882352941176471,
  'recall': 0.78125,
  'f1-score': 0.6711409395973155,
  'support': 64},
 '1': {'precision': 0.7142857142857143,
  'recall': 0.5,
  'f1-score': 0.588235294117647,
  'support': 180},
 '2': {'precision': 0.6385542168674698,
  'recall': 0.726027397260274,
  'f1-score': 0.6794871794871795,
  'support': 73},
 '3': {'precision': 0.70625,
  'recall': 0.6312849162011173,
  'f1-score': 0.6666666666666666,
  'support': 179},
 '4': {'precision': 0.5522388059701493,
  'recall': 0.6379310344827587,
  'f1-score': 0.5920000000000001,
  'support': 58},
 '5': {'precision': 0.8897058823529411,
  'recall': 0.5734597156398105,
  'f1-score': 0.6974063400576368,
  'support': 211},
 '6': {'precision': 0.3076923076923077,
  'recall': 0.48,
  'f1-score': 0.37500000000000006,
  'support': 25},
 '7': {'precision': 0.9391304347826087,
  'recall': 0.7248322147651006,
  'f1-score': 0.8181818181818182,
  'support': 149},
 '8': {'precision': 0.21875,
  'recall': 0.4242424242424